In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs
th.backends.cudnn.benchmark = True

In [ ]:

train_inputs = [th.randn(80, 2, 64, 1, device='cpu') * 1e-4 for _ in range(2)]
n_split = len(train_inputs[0]) - 40
test_inputs = [t[-20:] for t in train_inputs]
train_inputs = [t[:-20] for t in train_inputs]

In [ ]:
cuda = True
if cuda:
    train_inputs = [i.cuda() for i in train_inputs]
    test_inputs = [i.cuda() for i in test_inputs]

In [ ]:
train_inputs[0].data[:] += 1
test_inputs[0].data[:] += 1


In [ ]:
from reversible2.graph import Node
from reversible2.branching import CatChans, ChunkChans, Select
from reversible2.constantmemory import sequential_to_constant_memory
from reversible2.constantmemory import graph_to_constant_memory
def invert(feature_model, out):
    return feature_model.invert(out)

from copy import deepcopy
from reversible2.graph import Node
from reversible2.distribution import TwoClassDist
from reversible2.wrap_invertible import WrapInvertible
from reversible2.blocks import dense_add_no_switch, conv_add_3x3_no_switch
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter

set_random_seeds(2019011641, cuda)
n_chans = train_inputs[0].shape[1]
n_time = train_inputs[0].shape[2]
base_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False), # 4 x 32
    conv_add_3x3_no_switch(2*n_chans,32),
    conv_add_3x3_no_switch(2*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 8 x 16
    conv_add_3x3_no_switch(4*n_chans,32),
    conv_add_3x3_no_switch(4*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 16 x 8
    conv_add_3x3_no_switch(8*n_chans,32),
    conv_add_3x3_no_switch(8*n_chans,32))
base_model.cuda();

branch_1_a =  nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False), # 16 x 4
    conv_add_3x3_no_switch(8*n_chans,32),
    conv_add_3x3_no_switch(8*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),# 32 x 2
    conv_add_3x3_no_switch(16*n_chans,32),
    conv_add_3x3_no_switch(16*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 64 x 1
    ViewAs((-1,64,1,1), (-1,64))
)
branch_1_b = deepcopy(branch_1_a)
branch_1_a.cuda();
branch_1_b.cuda();



final_model = nn.Sequential(
    dense_add_no_switch(n_time*n_chans,256),
    dense_add_no_switch(n_time*n_chans,256),
    dense_add_no_switch(n_time*n_chans,256),
    dense_add_no_switch(n_time*n_chans,256),
)
final_model.cuda();
o = Node(None, base_model)
o = Node(o, ChunkChans(2))
o1a = Node(o, Select(0))
o1b = Node(o, Select(1))
o1a = Node(o1a, branch_1_a)
o1b = Node(o1b, branch_1_b)
o = Node([o1a,o1b,], CatChans())
o = Node(o, final_model)
#o = graph_to_constant_memory(o)
feature_model = o
if cuda:
    feature_model.cuda()
feature_model.eval();

In [ ]:
data_zero_init = True
set_distribution_to_empirical = True

In [ ]:
from reversible2.constantmemory import clear_ctx_dicts
from reversible2.distribution import TwoClassDist

if data_zero_init:
    feature_model.data_init(th.cat((train_inputs[0], train_inputs[1]), dim=0))

# Check that forward + inverse is really identical
t_out = feature_model(train_inputs[0][:2])
inverted = invert(feature_model, t_out)
clear_ctx_dicts(feature_model)
assert th.allclose(train_inputs[0][:2], inverted, rtol=1e-3,atol=1e-4)
device = list(feature_model.parameters())[0].device
from reversible2.ot_exact import ot_euclidean_loss_for_samples
class_dist = TwoClassDist(2, np.prod(train_inputs[0].size()[1:]) - 2, [0,1])
class_dist.cuda()

if set_distribution_to_empirical:
    for i_class in range(2):
        with th.no_grad():
            this_outs = feature_model(train_inputs[i_class])
            mean = th.mean(this_outs, dim=0)
            std = th.std(this_outs, dim=0)
            class_dist.set_mean_std(i_class, mean, std)
            # Just check
            setted_mean, setted_std = class_dist.get_mean_std(i_class)
            assert th.allclose(mean, setted_mean)
            assert th.allclose(std, setted_std)
    clear_ctx_dicts(feature_model)

optim_model = th.optim.Adam(feature_model.parameters(), lr=1e-3, betas=(0.9,0.999))
optim_dist = th.optim.Adam(class_dist.parameters(), lr=1e-2, betas=(0.9,0.999))


In [ ]:
def compute_accs(feature_model, train_inputs, test_inputs, class_dist):
    with th.no_grad():
        # Compute dist for mean/std of encodings
        data_cls_dists = []
        for i_class in range(len(train_inputs)):
            this_class_outs = feature_model(train_inputs[i_class])[:, class_dist.i_class_inds]
            data_cls_dists.append(
                th.distributions.MultivariateNormal(th.mean(this_class_outs, dim=0),
                covariance_matrix=th.diag(th.std(this_class_outs, dim=0) ** 2)))
        results = {}
        for setname, set_inputs in (("Train", train_inputs), ("Test", test_inputs)):
            outs = [feature_model(ins) for ins in set_inputs]
            c_outs = [o[:, class_dist.i_class_inds] for o in outs]

            c_outs_all = th.cat(c_outs)

            cls_dists = []
            for i_class in range(len(c_outs)):
                mean, std = class_dist.get_mean_std(i_class)
                cls_dists.append(
                    th.distributions.MultivariateNormal(mean[class_dist.i_class_inds],
                                                        covariance_matrix=th.diag(std[class_dist.i_class_inds] ** 2)))

            preds_per_class = [th.stack([cls_dists[i_cls].log_prob(c_out)
                             for i_cls in range(len(cls_dists))],
                            dim=-1) for c_out in c_outs]

            pred_labels_per_class = [np.argmax(var_to_np(preds), axis=1)
                           for preds in preds_per_class]

            labels = np.concatenate([np.ones(len(set_inputs[i_cls])) * i_cls 
             for i_cls in range(len(train_inputs))])

            acc = np.mean(labels == np.concatenate(pred_labels_per_class))

            data_preds_per_class = [th.stack([data_cls_dists[i_cls].log_prob(c_out)
                             for i_cls in range(len(cls_dists))],
                            dim=-1) for c_out in c_outs]
            data_pred_labels_per_class = [np.argmax(var_to_np(data_preds), axis=1)
                                for data_preds in data_preds_per_class]
            data_acc = np.mean(labels == np.concatenate(data_pred_labels_per_class))
            results['{:s}_acc'.format(setname.lower())] = acc
            results['{:s}_data_acc'.format(setname.lower())] = data_acc
    return results

In [ ]:
import pandas as pd
df = pd.DataFrame()

from reversible2.training import OTTrainer
trainer = OTTrainer(feature_model, class_dist,
                optim_model, optim_dist)

In [ ]:
from reversible2.constantmemory import clear_ctx_dicts
from reversible2.timer import Timer
from plot import plot_outs
from reversible2.gradient_penalty import gradient_penalty


i_start_epoch_out = 401
n_epochs = 1001
gen_frequency = 10
for i_epoch in range(n_epochs):
    epoch_row = {}
    with Timer(name='EpochLoop', verbose=False) as loop_time:
        gen_update = (i_epoch % gen_frequency) == (gen_frequency-1)
        loss_on_outs = i_epoch >= i_start_epoch_out
        result = trainer.train(train_inputs, loss_on_outs=loss_on_outs)
        
    epoch_row.update(result)
    epoch_row['runtime'] = loop_time.elapsed_secs * 1000
    acc_results = compute_accs(feature_model, train_inputs, test_inputs, class_dist)
    epoch_row.update(acc_results)
    if i_epoch % (n_epochs // 20) != 0:
        df = df.append(epoch_row, ignore_index=True)
        # otherwise add ot loss in
    else:
        for i_class in range(len(train_inputs)):
            with th.no_grad():
                class_ins = train_inputs[i_class]
                samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * 4)
                inverted = feature_model.invert(samples)
                clear_ctx_dicts(feature_model)
                ot_loss_in = ot_euclidean_loss_for_samples(class_ins.view(class_ins.shape[0], -1),
                                                           inverted.view(inverted.shape[0], -1)[:(len(class_ins))])
                epoch_row['ot_loss_in_{:d}'.format(i_class)] = ot_loss_in.item()
        df = df.append(epoch_row, ignore_index=True)
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loop Time: {:.0f} ms".format(loop_time.elapsed_secs * 1000))
        display(df.iloc[-3:])
    
